In [1]:
import geopandas as gpd
import rasterio
from rasterio import features
import pandas as pd
import numpy as np
import os
from shapely.geometry import LineString
from pyproj import CRS


os.chdir('/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/Data')

flowlines=gpd.read_file('flowlines_with_pop_density.geojson')
dem = rasterio.open('output_USGS30m.tif')

In [2]:
# Ensure both datasets are in the same CRS
dem_crs = CRS(dem.crs)

if flowlines.crs != dem_crs:
    flowlines = flowlines.to_crs(dem_crs)

def sample_elevations_along_line(line_geom, dem, num_samples=100):
    """
    Samples elevations along a line geometry from a DEM raster.
    
    Parameters:
    - line_geom: Shapely LineString geometry of the flowline.
    - dem: Opened rasterio DEM object.
    - num_samples: Number of points to sample along the line.

    Returns:
    - elevations: List of elevation values sampled from the DEM.
    """
    # Generate equally spaced points along the line
    distances = np.linspace(0, line_geom.length, num_samples)
    points = [line_geom.interpolate(distance) for distance in distances]
    
    # Get the coordinates of the points
    coords = [(point.x, point.y) for point in points]
    
    # Sample the DEM at these points
    elevations = []
    for coord in coords:
        try:
            # Sample the DEM at the coordinate
            elevation = list(dem.sample([coord]))[0][0]
            if dem.nodata is not None and elevation == dem.nodata:
                elevations.append(None)
            else:
                elevations.append(elevation)
        except Exception:
            elevations.append(None)
    
    # Filter out None or NaN values
    elevations = [e for e in elevations if e is not None and not np.isnan(e)]
    return elevations

# Prepare a list to store the results
results = []

for idx, row in flowlines.iterrows():
    line_geom = row.geometry
    
    # Sample elevations along the line
    elevations = sample_elevations_along_line(line_geom, dem)
    
    # Calculate statistics if we have valid elevations
    if elevations:
        max_elev = max(elevations)
        min_elev = min(elevations)
        avg_elev = sum(elevations) / len(elevations)
    else:
        max_elev = min_elev = avg_elev = None
    
    # Store the original index, plus your stats
    results.append({
        'flow_idx': idx,            # The original index from flowlines
        'Max_Elevation': max_elev,
        'Min_Elevation': min_elev,
        'Avg_Elevation': avg_elev
    })

# Create a DataFrame from the results
elevation_df = pd.DataFrame(results)

# Make 'flow_idx' the index
elevation_df.set_index('flow_idx', inplace=True)

# Export the results to an Excel file
# Replace 'flowline_elevations.xlsx' with your desired output file name
elevation_df.to_excel('flowline_elevations.xlsx', index=False)

print("Elevation statistics calculated and saved to 'flowline_elevations.xlsx'.")

Elevation statistics calculated and saved to 'flowline_elevations.xlsx'.


In [3]:
flowlines_merged = flowlines.join(elevation_df, how='left')

In [4]:
flowlines_merged.head()

,operator_number,flowline_id,location_id,status,location_type,fluid,material,diameter_in,length_ft,max_operating_pressure,shape_length,line_age_yr,root_cause,risk,risk_probability,average_pop_density,geometry,Max_Elevation,Min_Elevation,Avg_Elevation
0,10110,470450.0,470449.0,Active,Production Facilities,Crude Oil,Steel,2,542,34,165,52,Unknown,1,1.0,3664.680384,"LINESTRING (-104.47065 39.8447, -104.47245 39....",1614.386353,1612.644043,1613.260254
1,69175,477981.0,447490.0,New Construction,Production Facilities,Other,Carbon Steel,3,404,2430,123,7,Unknown,1,1.0,3664.680384,"LINESTRING (-104.65739 40.36763, -104.65647 40...",1419.969849,1419.780396,1419.840332
2,47120,457300.0,318070.0,Active,Production Facilities,Produced Water,Steel,2,18,2030,5,13,Unknown,1,1.0,3664.680384,"LINESTRING (-104.68709 40.15947, -104.68709 40...",1488.291382,1488.250488,1488.272827
3,100322,457931.0,422528.0,Active,Production Facilities,Multiphase,Carbon Steel,2,1135,435,345,20,Unknown,1,1.0,3664.680384,"LINESTRING (-104.58882 40.32276, -104.58707 40...",1472.394531,1464.031372,1468.571411
4,100322,466186.0,455178.0,Abandoned,Production Facilities,Multiphase,Carbon Steel,2,768,1006,234,31,Unknown,1,1.0,3664.680384,"LINESTRING (-104.74646 40.49752, -104.74466 40...",1472.549927,1469.974487,1470.980591


In [5]:
flowlines_merged.to_file("flowlines_pop_dems.geojson", driver='GeoJSON')